In [1]:
import requests
import pandas as pd

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import os

# Your Census API key
api_key = os.getenv("api_key")

In [ ]:

base_url = 'https://api.census.gov/data/{}/acs/acs5'

variables = {
    'total_population': 'B01003_001E',
    'white_population': 'B02001_002E',
    'black_population': 'B02001_003E',
    'american_indian_population': 'B02001_004E',
    'asian_population': 'B02001_005E',
    'native_hawaiian_other_pacific_islander_population': 'B02001_006E',
    'some_other_race_population': 'B02001_007E',
    'two_or_more_races_population': 'B02001_008E',
    'hispanic_or_latino_population': 'B03001_003E',
    'college_educated': 'B15003_022E',
    'median_income': 'B19013_001E',
    'poverty_status': 'B17001_001E',
    'under_poverty': 'B17001_002E'
}

years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
all_data = []

for year in years:
    for var, var_code in variables.items():
        # Fetch data for all counties in all states
        url = f'{base_url.format(year)}?get={var_code}&for=county:*&key={api_key}'
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                headers = data[0]
                values = data[1:]
                df = pd.DataFrame(values, columns=headers)
                df['year'] = year
                df['variable'] = var
                all_data.append(df)
            else:
                print(f"Failed to fetch data for {var} in {year}: {response.status_code}")
                print(response.text)
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")

combined_data = pd.concat(all_data, ignore_index=True)
combined_data.to_csv('census_data_all_states_counties.csv', index=False)

In [19]:
variables = {
    'total_population': 'B01003_001E',
    'white_population': 'B02001_002E',
    'black_population': 'B02001_003E',
    'american_indian_population': 'B02001_004E',
    'asian_population': 'B02001_005E',
    'native_hawaiian_other_pacific_islander_population': 'B02001_006E',
    'some_other_race_population': 'B02001_007E',
    'two_or_more_races_population': 'B02001_008E',
    'hispanic_or_latino_population': 'B03001_003E',
    'college_educated': 'B15003_022E',
    'median_income': 'B19013_001E',
    'poverty_status': 'B17001_001E',
    'under_poverty': 'B17001_002E'
}

years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
all_data = []

for year in years:
    query_vars = ','.join(variables.values())
    url = f"{base_url.format(year)}?get={query_vars},NAME&for=county:*&in=state:*&key={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            headers = data[0] + ['year']  # Append 'year' to the list of column headers
            values = [row + [year] for row in data[1:]]  # Append the year to each row of data
            df = pd.DataFrame(values, columns=headers)
            all_data.append(df)
        else:
            print(f"Failed to fetch data for {year}: {response.status_code}")
            print(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except Exception as e:
        print(f"Error: {e}")

if all_data:
    # Combine all collected data
    combined_data = pd.concat(all_data, ignore_index=True)

    # Create a new DataFrame that maps each variable to its name
    variable_names = {v: k for k, v in variables.items()}
    combined_data = combined_data.rename(columns=variable_names)

    # Map 'NAME' to 'location_name' and extract 'state' and 'county' from it
    combined_data['location_name'] = combined_data['NAME']
    combined_data[['state_name', 'county_name']] = combined_data['location_name'].str.split(',', expand=True)
    combined_data.drop(columns=['NAME'], inplace=True)

    # Pivot the DataFrame so each row is a unique combination of state, county, and year with all variables as columns
    pivoted_data = combined_data.pivot_table(index=['state', 'county', 'state_name', 'county_name', 'year'], aggfunc='first').reset_index()

    # Save the restructured data to CSV
    pivoted_data.to_csv('census_data_all_states_counties_restructured.csv', index=False)
else:
    print("No data collected.")

In [3]:




import requests
import pandas as pd


# Define the base URL for the Census API
base_url = 'https://api.census.gov/data/'

# Function to fetch data from the Census API
def fetch_census_data(year, geographic_level='county'):
    if year == 2010:
        dataset = 'dec/sf1'
        fields = ['P001001', 'P002005', 'P002002']  # total pop, Rural population, Urban population
    elif year == 2020:
        dataset = 'dec/pl'
        fields = ['P1_001N', 'P2_005N', 'P2_002N']  # total pop, Urban population, Rural population
    else:
        raise ValueError('Invalid year. Only 2010 and 2020 are supported.')
    
    if geographic_level == 'county':
        for_clause = 'county:*'
    elif geographic_level == 'state':
        for_clause = 'state:*'
    else:
        raise ValueError('Invalid geographic level. Only "county" and "state" are supported.')

    params = {
        'get': ','.join(fields),
        'for': for_clause,
        'key': api_key
    }

    response = requests.get(base_url + str(year) + '/' + dataset, params=params)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        # Calculating percentage of urban and rural population
        if year == 2010:
            df['rural_pop_percent'] = df['P002005'].astype(int) / df['P001001'].astype(int) * 100
            df['urban_pop_percent'] = df['P002002'].astype(int) / df['P001001'].astype(int) * 100
        elif year == 2020:
            df['rural_pop_percent'] = df['P2_005N'].astype(int) / df['P1_001N'].astype(int) * 100
            df['urban_pop_percent'] = df['P2_002N'].astype(int) / df['P1_001N'].astype(int) * 100
        return df
    else:
        response.raise_for_status()

# Fetch data for 2010 and 2020
census_2010 = fetch_census_data(2010, 'county')
census_2020 = fetch_census_data(2020, 'county')

# Display the first few rows of the data
print("2010 Census Data:")
print(census_2010.head())

print("\n2020 Census Data:")
print(census_2020.head())

# Save data to CSV files
census_2010.to_csv('census_2010_county.csv', index=False)
census_2020.to_csv('census_2020_county.csv', index=False)

2010 Census Data:
  P001001 P002005 P002002 state county  rural_pop_percent  urban_pop_percent
0  125744   26170   99574    05    131          20.812126          79.187874
1   17058   10849    6209    05    133          63.600657          36.399343
2   17264   13821    3443    05    135          80.056766          19.943234
3   12394   12394       0    05    137         100.000000           0.000000
4   41639   22695   18944    05    139          54.504191          45.495809

2020 Census Data:
  P1_001N P2_005N P2_002N state county  rural_pop_percent  urban_pop_percent
0   58805   41582    2117    01    001          70.711674           3.600034
1  231767  186495   12686    01    003          80.466589           5.473601
2   25223   11086    1510    01    005          43.951949           5.986600
3   22293   16442     740    01    007          73.754093           3.319428
4   59134   49764    5771    01    009          84.154632           9.759191


In [20]:
# Convert columns to numeric type
cols_to_convert = ['under_poverty', 'poverty_status', 'white_population', 'total_population',
                   'college_educated', 'black_population', 'hispanic_or_latino_population',
                  'american_indian_population', 'asian_population','native_hawaiian_other_pacific_islander_population',
                  'two_or_more_races_population']

for col in cols_to_convert:
    pivoted_data[col] = pd.to_numeric(pivoted_data[col], errors='coerce')

    
# Calculate the percentage of persons below poverty
pivoted_data['percentage_of_persons_below_poverty'] = (pivoted_data['under_poverty'] / pivoted_data['poverty_status']) * 100

# Calculate the percentage of white
pivoted_data['percentage_of_white'] = (pivoted_data['white_population'] / pivoted_data['total_population']) * 100

# Calculate the percentage of college educated
pivoted_data['percentage_of_college_educated'] = (pivoted_data['college_educated'] / pivoted_data['total_population']) * 100

# Calculate the percentage of black
pivoted_data['percentage_of_black'] = (pivoted_data['black_population'] / pivoted_data['total_population']) * 100

# Calculate the percentage of Hispanics and Latinos
pivoted_data['percentage_of_hispanics_and_latinos'] = (pivoted_data['hispanic_or_latino_population'] / pivoted_data['total_population']) * 100


# Calculate the percentage of American Indian
pivoted_data['percentage_of_american_indian_population'] = (pivoted_data['american_indian_population'] / pivoted_data['total_population']) * 100

# Calculate the percentage of Asian 
pivoted_data['percentage_of_asian_population'] = (pivoted_data['asian_population'] / pivoted_data['total_population']) * 100

# Calculate the percentage of Native Hawaaiian and other Pacific Islanders
pivoted_data['percentage_of_native_hawaiian_other_pacific_islander_population'] = (pivoted_data['native_hawaiian_other_pacific_islander_population'] / pivoted_data['total_population']) * 100

# Calculate the percentage of 2 or more races
pivoted_data['percentage_of_two_or_more_races_population'] = (pivoted_data['two_or_more_races_population'] / pivoted_data['two_or_more_races_population']) * 100


# Save the modified DataFrame
pivoted_data.to_csv('updated_census_data_all_states_counties_restructured.csv', index=False)

In [26]:
import pandas as pd

# Load the CSV file
file_path = 'County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
data = pd.read_csv(file_path)

# Display the first few rows and columns of the data
data.head(), data.columns


(   RegionID  SizeRank          RegionName RegionType StateName State  \
 0      3101         0  Los Angeles County     county        CA    CA   
 1       139         1         Cook County     county        IL    IL   
 2      1090         2       Harris County     county        TX    TX   
 3      2402         3     Maricopa County     county        AZ    AZ   
 4      2841         4    San Diego County     county        CA    CA   
 
                                   Metro  StateCodeFIPS  MunicipalCodeFIPS  \
 0    Los Angeles-Long Beach-Anaheim, CA              6                 37   
 1    Chicago-Naperville-Elgin, IL-IN-WI             17                 31   
 2  Houston-The Woodlands-Sugar Land, TX             48                201   
 3             Phoenix-Mesa-Chandler, AZ              4                 13   
 4    San Diego-Chula Vista-Carlsbad, CA              6                 73   
 
       2000-01-31  ...     2023-08-31     2023-09-30     2023-10-31  \
 0  215786.073863  

In [27]:
# Define the columns to keep
columns_to_keep = [
    "RegionID", "RegionName", "State", "Metro", "StateCodeFIPS", "MunicipalCodeFIPS",
    "2012-12-31", "2013-12-31", "2014-12-31", "2015-12-31", "2016-12-31",
    "2017-12-31", "2018-12-31", "2019-12-31",  "2020-12-31", "2021-12-31", "2022-12-31"
]

# Filter the dataset to include only the specified columns
filtered_data = data[columns_to_keep]

# Display the first few rows of the filtered data
filtered_data.head()


,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
0,3101,Los Angeles County,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,380343.017170,450607.320815,488525.891318,481342.429439,509799.989998,565053.183056,655649.269498,659663.889122,721301.627317,815926.505864,859839.347975
1,139,Cook County,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,158319.469896,179575.598141,194267.993514,204182.716598,214897.711437,226380.549162,234063.718033,236432.646792,250751.247704,273135.948432,285309.653910
2,1090,Harris County,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,126895.639870,140329.279074,156916.290625,171963.065168,181116.149705,188358.539565,198382.623110,206091.278897,219493.450422,253580.231771,287212.997987
3,2402,Maricopa County,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,169415.307487,203154.981315,212563.892982,231968.753259,247212.354942,263995.775842,284438.340213,301515.213522,345326.918380,444646.503853,472283.523460
4,2841,San Diego County,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,366173.240425,434539.740946,463246.999457,502928.753297,535880.753701,579358.122526,600516.898695,611252.645258,673409.699348,793021.912618,863462.051965


In [28]:
# Rename the date columns to only keep the year information
filtered_data.columns = [
    "RegionID", "RegionName", "State", "Metro", "StateCodeFIPS", "MunicipalCodeFIPS",
    "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"
]

# Display the first few rows to confirm the column renaming
filtered_data.head()


,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,3101,Los Angeles County,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,380343.017170,450607.320815,488525.891318,481342.429439,509799.989998,565053.183056,655649.269498,659663.889122,721301.627317,815926.505864,859839.347975
1,139,Cook County,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,158319.469896,179575.598141,194267.993514,204182.716598,214897.711437,226380.549162,234063.718033,236432.646792,250751.247704,273135.948432,285309.653910
2,1090,Harris County,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,126895.639870,140329.279074,156916.290625,171963.065168,181116.149705,188358.539565,198382.623110,206091.278897,219493.450422,253580.231771,287212.997987
3,2402,Maricopa County,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,169415.307487,203154.981315,212563.892982,231968.753259,247212.354942,263995.775842,284438.340213,301515.213522,345326.918380,444646.503853,472283.523460
4,2841,San Diego County,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,366173.240425,434539.740946,463246.999457,502928.753297,535880.753701,579358.122526,600516.898695,611252.645258,673409.699348,793021.912618,863462.051965


In [29]:
# Load the census data
census_data_path = 'updated_census_data_all_states_counties_restructured.csv'
census_data = pd.read_csv(census_data_path)

# Display the first few rows and columns of the census data
census_data.head(), census_data.columns


(   state  county      state_name county_name  year  \
 0      1       1  Autauga County     Alabama  2012   
 1      1       1  Autauga County     Alabama  2013   
 2      1       1  Autauga County     Alabama  2014   
 3      1       1  Autauga County     Alabama  2015   
 4      1       1  Autauga County     Alabama  2016   
 
    american_indian_population  asian_population  black_population  \
 0                         169               439              9880   
 1                         138               525             10076   
 2                         230               500             10257   
 3                         230               534             10315   
 4                         226               444             10196   
 
    college_educated  hispanic_or_latino_population  ... white_population  \
 0              5085                           1310  ...            43084   
 1              4899                           1365  ...            42997   
 2             

In [30]:
# Pivot the census data for the 'total_population' column as an example
pivot_example = census_data.pivot_table(
    values='total_population', 
    index=['state', 'county', 'state_name', 'county_name'], 
    columns='year',
    aggfunc='first'
)

# Display the pivoted data
pivot_example.head()


year                                         2012      2013      2014  \
state county state_name     county_name                                 
1     1      Autauga County  Alabama      54590.0   54907.0   55136.0   
      3      Baldwin County  Alabama     183226.0  187114.0  191205.0   
      5      Barbour County  Alabama      27469.0   27321.0   27119.0   
      7      Bibb County     Alabama      22769.0   22754.0   22653.0   
      9      Blount County   Alabama      57466.0   57623.0   57645.0   

year                                         2015      2016      2017  \
state county state_name     county_name                                 
1     1      Autauga County  Alabama      55221.0   55049.0   55036.0   
      3      Baldwin County  Alabama     195121.0  199510.0  203360.0   
      5      Barbour County  Alabama      26932.0   26614.0   26201.0   
      7      Bibb County     Alabama      22604.0   22572.0   22580.0   
      9      Blount County   Alabama      57710.0   57704.0   57667.0   

year                                         2018      2019      2020  \
state county state_name     county_name                                 
1     1      Autauga County  Alabama      55200.0   55380.0   55639.0   
      3      Baldwin County  Alabama     208107.0  212830.0  218289.0   
      5      Barbour County  Alabama      25782.0   25361.0   25026.0   
      7      Bibb County     Alabama      22527.0   22493.0   22374.0   
      9      Blount County   Alabama      57645.0   57681.0   57755.0   

year                                         2021      2022  
state county state_name     county_name                      
1     1      Autauga County  Alabama      58239.0   58761.0  
      3      Baldwin County  Alabama     227131.0  233420.0  
      5      Barbour County  Alabama      25259.0   24877.0  
      7      Bibb County     Alabama      22412.0   22251.0  
      9      Blount County   Alabama      58884.0   59077.0

In [31]:
# List of columns to pivot apart from 'total_population'
columns_to_pivot = [
    'total_population','american_indian_population', 'asian_population', 'black_population', 
    'college_educated', 'hispanic_or_latino_population', 'median_income',
    'native_hawaiian_other_pacific_islander_population', 'poverty_status',
    'some_other_race_population', 'two_or_more_races_population', 'under_poverty',
    'white_population', 'percentage_of_persons_below_poverty', 'percentage_of_white',
    'percentage_of_college_educated', 'percentage_of_black', 'percentage_of_hispanics_and_latinos',
    'percentage_of_american_indian_population', 'percentage_of_asian_population',
    'percentage_of_native_hawaiian_other_pacific_islander_population', 'percentage_of_two_or_more_races_population'
]

# Pivot each demographic column while appending the year to the column name during the pivot process
pivoted_census_data_with_year = pd.DataFrame(index=census_data.set_index(['state', 'county', 'state_name', 'county_name']).index.unique())

# Loop through each column to pivot and add to the dataframe with year-suffixed column names
for column in columns_to_pivot:
    pivoted_column = census_data.pivot_table(
        values=column, 
        index=['state', 'county', 'state_name', 'county_name'], 
        columns='year',
        aggfunc='first'
    )
    # Rename the pivoted columns to include the original demographic name with the year
    pivoted_column.columns = [f"{column}_{year}" for year in pivoted_column.columns]
    pivoted_census_data_with_year = pd.concat([pivoted_census_data_with_year, pivoted_column], axis=1)

# Reset index to make state and county as separate columns for the merge
pivoted_census_data_with_year.reset_index(inplace=True)

# Display the first few rows of the fully pivoted data with year-suffixed column names
pivoted_census_data_with_year.head()


,state,county,state_name,county_name,total_population_2012,total_population_2013,total_population_2014,total_population_2015,total_population_2016,total_population_2017,...,percentage_of_two_or_more_races_population_2013,percentage_of_two_or_more_races_population_2014,percentage_of_two_or_more_races_population_2015,percentage_of_two_or_more_races_population_2016,percentage_of_two_or_more_races_population_2017,percentage_of_two_or_more_races_population_2018,percentage_of_two_or_more_races_population_2019,percentage_of_two_or_more_races_population_2020,percentage_of_two_or_more_races_population_2021,percentage_of_two_or_more_races_population_2022
0,1,1,Autauga County,Alabama,54590.0,54907.0,55136.0,55221.0,55049.0,55036.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,1,3,Baldwin County,Alabama,183226.0,187114.0,191205.0,195121.0,199510.0,203360.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,1,5,Barbour County,Alabama,27469.0,27321.0,27119.0,26932.0,26614.0,26201.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,1,7,Bibb County,Alabama,22769.0,22754.0,22653.0,22604.0,22572.0,22580.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,1,9,Blount County,Alabama,57466.0,57623.0,57645.0,57710.0,57704.0,57667.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [32]:
# Ensure the data types match for merging
pivoted_census_data_with_year['state'] = pivoted_census_data_with_year['state'].astype(int)
pivoted_census_data_with_year['county'] = pivoted_census_data_with_year['county'].astype(int)
filtered_data['StateCodeFIPS'] = filtered_data['StateCodeFIPS'].astype(int)
filtered_data['MunicipalCodeFIPS'] = filtered_data['MunicipalCodeFIPS'].astype(int)

# Merge the datasets
merged_data = pd.merge(
    filtered_data,
    pivoted_census_data_with_year,
    left_on=['StateCodeFIPS', 'MunicipalCodeFIPS'],
    right_on=['state', 'county'],
    how='left'
)

# Display the first few rows of the merged data
merged_data.head()


/var/folders/jf/qg8srjzs7d9bg9npqyhh8x1c0000gn/T/ipykernel_1563/2259310655.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['StateCodeFIPS'] = filtered_data['StateCodeFIPS'].astype(int)
/var/folders/jf/qg8srjzs7d9bg9npqyhh8x1c0000gn/T/ipykernel_1563/2259310655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['MunicipalCodeFIPS'] = filtered_data['MunicipalCodeFIPS'].astype(int)


,RegionID,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2012,2013,2014,2015,...,percentage_of_two_or_more_races_population_2013,percentage_of_two_or_more_races_population_2014,percentage_of_two_or_more_races_population_2015,percentage_of_two_or_more_races_population_2016,percentage_of_two_or_more_races_population_2017,percentage_of_two_or_more_races_population_2018,percentage_of_two_or_more_races_population_2019,percentage_of_two_or_more_races_population_2020,percentage_of_two_or_more_races_population_2021,percentage_of_two_or_more_races_population_2022
0,3101,Los Angeles County,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,380343.017170,450607.320815,488525.891318,481342.429439,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,139,Cook County,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,158319.469896,179575.598141,194267.993514,204182.716598,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,1090,Harris County,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,126895.639870,140329.279074,156916.290625,171963.065168,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,2402,Maricopa County,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,169415.307487,203154.981315,212563.892982,231968.753259,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,2841,San Diego County,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,366173.240425,434539.740946,463246.999457,502928.753297,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [33]:
# Define the path for the resulting CSV file
output_csv_path = 'merged_dataset.csv'

# Save the merged data to a CSV file
merged_data.to_csv(output_csv_path, index=False)

# Provide the path for download
output_csv_path


'merged_dataset.csv'

In [34]:
# Rename the columns with years only to "home_value_<year>"
year_columns = [str(year) for year in range(2012, 2023)]
new_column_names = {year: f"home_value_{year}" for year in year_columns}
merged_data_renamed = merged_data.rename(columns=new_column_names)

# Display the updated column names to confirm the changes
merged_data_renamed.columns


Index(['RegionID', 'RegionName', 'State', 'Metro', 'StateCodeFIPS',
       'MunicipalCodeFIPS', 'home_value_2012', 'home_value_2013',
       'home_value_2014', 'home_value_2015',
       ...
       'percentage_of_two_or_more_races_population_2013',
       'percentage_of_two_or_more_races_population_2014',
       'percentage_of_two_or_more_races_population_2015',
       'percentage_of_two_or_more_races_population_2016',
       'percentage_of_two_or_more_races_population_2017',
       'percentage_of_two_or_more_races_population_2018',
       'percentage_of_two_or_more_races_population_2019',
       'percentage_of_two_or_more_races_population_2020',
       'percentage_of_two_or_more_races_population_2021',
       'percentage_of_two_or_more_races_population_2022'],
      dtype='object', length=263)

In [35]:
# Define the path for the resulting CSV file
output_csv_path = 'merged_dataset.csv'

# Save the merged data to a CSV file
merged_data_renamed.to_csv(output_csv_path, index=False)

# Provide the path for download
output_csv_path


'merged_dataset.csv'